### Prototype code to scrape the top headlines and their corresponding article text from News API

In [1]:
import os.path
from os import path
import pandas as pd
import json

In [2]:
import pprint
import requests

In [3]:
# API Key for News API
secret = 'd05c3ae08abd461f90b3dc555af1bb59'

In [4]:
# Define the endpoint to extract all the top headlines 
url = 'https://newsapi.org/v2/top-headlines?'

In [5]:
# Specify the query and number of returns - Limit the headlines to country US, for now
parameters = {
    'language': 'en',
    'country':'us',
    'pageSize': 100,
    'apiKey': secret 
}

In [6]:
categories = ['business','entertainment','general','health','science','sports','technology']


In [7]:
from newspaper import Article
from newspaper import Config


article_title = []
article_authors = []
article_text = []
article_summary = []
article_date = []
article_top_image = []
failed_url = []
category_articles ={}
pages= range(1,2)

In [8]:
#category_dataframe = pd.read_json(f'resources/business_top_headline_data.json').T
#category_dataframe.head()

In [ ]:
count =0
for category in categories:
    name = category
    print(name)
    bappend = False
    if os.path.exists(f'resources/{category}_top_headline_data.json') :
        category_dataframe = pd.read_json(f'resources/{category}_top_headline_data.json').T
        print(f"Using DataFrame {category_dataframe.shape}" )
        bappend = True
    catDF= None
    parameters['category'] = category
    response = requests.get(url, params=parameters)
    if response.status_code != requests.codes.ok:
        print(f"Bad result : {response.url}")
        continue
    response_json = response.json()

    df = pd.DataFrame.from_dict(response_json)
    df = pd.concat([df.drop(['articles'], axis=1), df['articles'].apply(pd.Series)], axis=1) 

    rows = []
    for index, row in df.iterrows():
        h = row['url']
        user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
        config = Config()
        config.verbose=True
        config.browser_user_agent = user_agent
        newsarticle = Article(h, config=config)
        try :
            newsarticle.download()
            newsarticle.parse()
            if not newsarticle.text :
                print(f"Unable to parse article: {h}")
                continue 
            artdict = {}                
            artdict['articleText']=newsarticle.text
            for column in df:
                artdict[column]= row[column]
            rows.append(artdict)
            count += 1
            if (count % 50 == 0 ):
                print('Number of articles: '+str(count))

        except :
             print(f'***FAILED TO DOWNLOAD***{newsarticle.url}')
             pass
    if name in category_articles:
        category_articles[name].append(pd.DataFrame(rows),ignore_index=True)
    else:
        if bappend :
            category_dataframe = category_dataframe.append(pd.DataFrame(rows),ignore_index=True).drop_duplicates(subset='url')
            category_articles[name] = category_dataframe
        else:
            category_articles[name]= pd.DataFrame(rows)
    print(category_articles[name].shape)


print('Total number of articles: '+str(count))
    

business
Using DataFrame (118, 11)
Unable to parse article: http://news3lv.com/news/local/clark-county-releases-names-of-businesses-found-in-violation-of-mask-mandate
Unable to parse article: http://13wham.com/news/local/local-malls-reopen-friday-but-some-stores-remain-closed
***FAILED TO DOWNLOAD***https://news.google.com/__i/rss/rd/articles/CBMiXWh0dHBzOi8vd3d3Lm5ld3N3ZWVrLmNvbS93YXlmYWlyLWNoaWxkLXRyYWZmaWNraW5nLWNvbnNwaXJhY3ktdGhlb3J5LWNhYmluZXRzLXNjYW5kYWwtMTUxNzAxM9IBY2h0dHBzOi8vd3d3Lm5ld3N3ZWVrLmNvbS93YXlmYWlyLWNoaWxkLXRyYWZmaWNraW5nLWNvbnNwaXJhY3ktdGhlb3J5LWNhYmluZXRzLXNjYW5kYWwtMTUxNzAxMz9hbXA9MQ?oc=5
***FAILED TO DOWNLOAD***https://news.google.com/__i/rss/rd/articles/CBMiTWh0dHBzOi8vd3d3LmFnd2ViLmNvbS9hcnRpY2xlL3dhc2RlLWxvd2VyLWNvcm4tc3VwcGxpZXMtZHVlLWFjcmVhZ2UtcmVkdWN0aW9u0gEA?oc=5
Unable to parse article: http://wgme.com/news/local/union-threatens-to-fine-biw-workers-if-they-cross-picket-line
Number of articles: 50
Unable to parse article: https://www.bizjournals.com/denver/

Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9WXc3QkR4THJQU1XSAQA?oc=5
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9SmI1VTQ1Y21LUjTSAQA?oc=5
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9a29OV2tpdU00enPSAQA?oc=5
***FAILED TO DOWNLOAD***https://www.washingtonpost.com/sports/2020/07/10/private-letter-redskins-fedex-said-it-will-remove-signage-if-name-isnt-changed/
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9azJ0TnVqR2JFVTDSAQA?oc=5
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9bmJIZEIxT1FzVjjSAQA?oc=5
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9eHUtS2hBWjRpWm_SAQA?

In [ ]:
for cat in category_articles.keys():
    print(cat)
    df = category_articles[cat]
    df.to_json(f'resources/{cat}_top_headline_data.json', orient='index')
    
    